# Clip v1_1 snow dynamics to HBL

In [1]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask

In [5]:
def clip_tifs_by_shapefile(
    # input_dir=r"F:\HLS\HBL\Products\snowDynamics_v_1_1\winterYear",
    # shapefile_path=r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\UtilityData\Study Areas\HBL\HBL_3979.shp",
    # output_dir=r"F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only"
    input_dir=r"F:\HLS\HBL\Products\snowDynamics_v_1_1\interannual\2018-2024",
    shapefile_path=r"C:\Users\mbonney\OneDrive - NRCan RNCan\Projects\UtilityData\Study Areas\HBL\HBL_3979.shp",
    output_dir=r"F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual"
):
    """
    Recursively clips all TIFF files in input_dir by the given shapefile
    and saves results to output_dir with NoData outside the shapefile.
    """

    # Load shapefile
    gdf = gpd.read_file(shapefile_path)
    shapes = gdf.geometry.values

    for root, _, files in os.walk(input_dir):
        for file in files:
            if not file.lower().endswith(".tif"):
                continue

            in_tif = os.path.join(root, file)

            # Preserve relative folder structure
            rel_path = os.path.relpath(root, input_dir)
            out_folder = os.path.join(output_dir, rel_path)
            os.makedirs(out_folder, exist_ok=True)

            out_tif = os.path.join(out_folder, file)

            with rasterio.open(in_tif) as src:
                # Reproject shapefile if needed
                if gdf.crs != src.crs:
                    gdf_proj = gdf.to_crs(src.crs)
                    shapes = gdf_proj.geometry.values

                clipped_data, clipped_transform = mask(
                    src,
                    shapes,
                    crop=True,
                    nodata=src.nodata
                )

                out_meta = src.meta.copy()
                out_meta.update({
                    "height": clipped_data.shape[1],
                    "width": clipped_data.shape[2],
                    "transform": clipped_transform,
                    "nodata": src.nodata
                })

                with rasterio.open(out_tif, "w", **out_meta) as dst:
                    dst.write(clipped_data)

            print(f"Clipped: {out_tif}")

In [6]:
clip_tifs_by_shapefile()

Clipped: F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual\.\HLS_Fmask_v1_1_pPerennialSnow_1824_HBLRoF.tif
Clipped: F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual\.\HLS_Fmask_v1_1_pSnowFree_1824_HBLRoF.tif
Clipped: F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual\.\HLS_Fmask_v1_1_snow_endB_mn_1824_HBLRoF.tif
Clipped: F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual\.\HLS_Fmask_v1_1_snow_endB_q_mn_1824_HBLRoF.tif
Clipped: F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual\.\HLS_Fmask_v1_1_snow_endB_u_mn_1824_HBLRoF.tif
Clipped: F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual\.\HLS_Fmask_v1_1_snow_endL_mn_1824_HBLRoF.tif
Clipped: F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual\.\HLS_Fmask_v1_1_snow_endL_q_mn_1824_HBLRoF.tif
Clipped: F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual\.\HLS_Fmask_v1_1_snow_endL_u_mn_1824_HBLRoF.tif
Clipped: F:\HLS\HBL\Products\snowDynamics_v_1_1\HBL_only\interannual\.\HL